fix me
- help
- abbreviations
- bibliography

- in comps families,  exlcude word itself
- declensions - colour columns 12 / 34 / 56 differently
- conjugations - colour row 123 / 456 / 789 differently
- conjugations - active sg active pl reflexive sg reflexive pl

- eccosystem
- - goldendict
- - google sheets feedback
- - basic website with install instructions, update instructions. new features etc. 
- - google drive download update





setup

In [59]:
import pandas as pd
import re
import json
import pickle
import os
from datetime import date

today = date.today()

df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep = "\t", dtype=str)
df = df.fillna("") 
df_length = df.shape[0]

roots_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/roots.csv", sep = "\t", dtype=str)
roots_df = roots_df.fillna("")
roots_df.replace(to_replace =".0", value = "", regex = True, inplace=True)
roots_df_length = roots_df.shape[0]

generate html & json

In [60]:
print("~" * 40)
print("generating html & json files")
print("~" * 40)

error_log = open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/errorlogs/exporter errorlog.txt", "w")

html_data_list = []
text_data_full = ""
text_data_concise = ""

for row in range(df_length): #df_length

# define variables
	pali = df.loc[row, "Pāli1"]
	if row % 5000 == 0 or row % df_length == 0:
		print(f"{row}/{df_length}\t{pali}")
	pali_ = "_" + re.sub(" ", "_", pali) 
	pali2 = df.loc[row, "Pāli2"]
	pali_clean = re.sub(" \d*$", "", pali)
	pos = df.loc[row, "POS"]
	grammar = df.loc[row, "Grammar"]
	neg = df.loc[row, "Neg"]
	verb = df.loc[row, "Verb"]
	trans = df.loc[row, "Trans"]
	case = df.loc[row, "Case"]
	meaning = df.loc[row, "Meaning IN CONTEXT"]
	lit = df.loc[row, "Literal Meaning"]
	buddhadatta = df.loc[row, "Buddhadatta"]
	non_ia = df.loc[row, "Non IA"]
	sk = df.loc[row, "Sanskrit"]
	sk_root = df.loc[row, "Sk Root"]
	sk_root_mn = df.loc[row, "Sk Root Mn"]
	sk_root_cl = df.loc[row, "Cl"]
	root = df.loc[row, "Pāli Root"]
	root_in_comps = df.loc[row, "Root In Comps"]
	root_verb = df.loc[row, "V"]
	root_grp = df.loc[row, "Grp"]
	root_sign = df.loc[row, "Sgn"]
	root_meaning = df.loc[row, "Root Meaning"]
	base = df.loc[row, "Base"]
	family = df.loc[row, "Family"]
	family2 = df.loc[row, "Family2"]
	construction =  df.loc[row, "Construction"]
	derivative = df.loc[row, "Derivative"]
	suffix = df.loc[row, "Suffix"]
	pc = df.loc[row, "Phonetic Changes"]
	comp = df.loc[row, "Compound"]
	comp_constr = df.loc[row, "Compound Construction"]
	source1 = df.loc[row, "Source1"]
	sutta1 = df.loc[row, "Sutta1"]
	eg1 = df.loc[row, "Example1"]
	source2 = df.loc[row, "Source 2"]
	sutta2 = df.loc[row, "Sutta2"]
	eg2 = df.loc[row, "Example 2"]
	ant = df.loc[row, "Antonyms"]
	syn = df.loc[row, "Synonyms – different word"]
	var = df.loc[row, "Variant – same constr or diff reading"]
	comm = df.loc[row, "Commentary"]
	comm = re.sub("(.+)\.$", "\\1", comm)
	notes = df.loc[row, "Notes"]
	cognate = df.loc[row, "Cognate"]
	category = df.loc[row, "Category"]
	metadata = df.loc[row, "Metadata"]
	link = df.loc[row, "Link"]

	colour1 = "#00A4CC" #dark blue
	colour2 = "#65DBFF" #inbetween for rollover
	colour3 = "#E2FFFF" #light blue

	root_colour0 = "#AB3C00" ##AB3C00"
	root_colour2 = "#F95700"
	root_colour3 = "#FFE2D2"

	x_colour0 = "#AB3C00"
	x_colour2 = "#F95700"
	x_colour3 = "#FFE2D2"

	indeclinables = ["abbrev", "abs", "ger", "ind", "inf", "prefix"]
	conjugations = ["aor", "cond", "fut", "imp", "imperf", "opt", "perf", "pr"]
	declensions = ["adj", "card", "cs", "fem", "letter", "masc", "nt", "ordin", "pp", "pron", "prp", "ptp", "root", "suffix", "ve"]

	html_string = ""
	text_full = ""
	text_concise = ""

# htm head & style

	html_string += f"""<!DOCTYPE html><html><head><meta charset="utf-8"></head><style>"""

	html_style = """
	
	body
	{ font-family : "Noto Sans", "DejaVu Serif", "Verajja Serif", "Crimson Pro" ;
	}
	
	p 
	{ line-height:170%;
	margin:5px 10px;
	vertical-align:middle;
	}

	table.table1 td, table.table1 th 
	{ border:1px solid #fff;
	line-height:150%;
	margin:5px 10px;
	padding:2px 5px;
	text-align:left;
	transition-duration: .2s;
	vertical-align:top;
	}

	table.table1 th 
	{ color: #00A4CC;
	font-weight:700;
	white-space:nowrap;
	}
	
	table.table1 td:hover, table.table1 th:hover
	{ background-color:#65DBFF;
	}
	
	table.dataframe td,table.dataframe th 
	{ background-color:#E2FFFF;
	border:1px solid #fff;
 	line-height:150%;
 	padding:5px;
 	text-align:center;
	 transition-duration: .2s;
 	vertical-align:middle;
	}
	
	table.dataframe th 
	{ background-color:#ffffff;
	color:#00A4CC;
	font-weight:700;
	}
	
	table.dataframe td:hover
	{ background-color:#65DBFF;
	}
	
	input 
	{ display:none;
	}

	div.content 
	{ border:2px solid #00A4CC;
	border-radius:5px;
	line-height:150%;
	overflow:auto;
	text-align:left;
	}

	label 
	{ background-color:#E2FFFF;
	border-radius:10px;
	box-shadow:0 4px 4px -2px #888888;
	color:#000;
	cursor:pointer;
	display:inline-block;
	font-size:80%;
	font-weight:400;
	margin:3px 0;
	padding:5px;
	transition-duration: .2s;
	}
	
	label:hover, input:checked + label:hover 
	{ background-color:#00A4CC;
	color:white;
	}

	.link 
	{ border-radius:10px;
	color:#00A4CC;
	font-weight:700;
	padding:5px 0;
	}

	input + label + div.content 
	{ display:none;
	}

	input:checked + label + div.content 
	{ display:block;
	}

	input:checked + label 
	{ background-color:#65DBFF;
	}
	"""

	html_string += f"""{html_style}</style><body>"""

# summary

	if meaning == "":
		html_string += f"""<div class="content"><p>{pos}. <b>{buddhadatta}</b> [under construction]</p></div>"""
		text_full += f"""{pali}. {grammar}. {buddhadatta}. [under construction]"""
		text_concise += f"""{pali}. {pos}. {buddhadatta}."""

	
	if meaning != "":
		html_string += f"""<div class="content"><p>{pos}"""
		text_concise += f"{pali}. {pos}"

		if case != "":
			html_string += f""" ({case})"""
			text_concise += f""" ({case})"""

		html_string += f""". <b>{meaning}.</b>"""
		text_concise += f""". {meaning}."""
		
		if base == "":
			construction_simple = re.sub(" \[.+\] \+", "", construction)
			construction_simple = re.sub("> .+? ", "", construction_simple)
			construction_simple = re.sub("<br/>.+", "", construction_simple)
			if construction_simple != "":
				html_string += f""" [{construction_simple}]"""
				text_concise += f""" [{construction_simple}]"""

		if base != "":
			family_plus = re.sub(" ", " + ", family)
			construction_oneline = re.sub("<br/>.+", "", construction) 
			construction_truncated = re.sub("(.+)(\+ .{1,7}$)", "\\2", construction_oneline)
			if re.match("^na ", construction):
				construction_na = re.sub("^(na )(.+)$", "\\1 + ", construction)
			else:
				construction_na = ""
			
			construction_reconstructed = f"{construction_na}{family_plus} + {root_sign} {construction_truncated}"
			html_string += f""" [{construction_reconstructed}]"""
			text_concise += f""" [{construction_reconstructed}]"""
		html_string += f"""</p></div>"""

# grammar
		html_string += f"""<input type="checkbox"  id="grammar{pali_}" /><label for="grammar{pali_}">grammar</label>"""
		html_string += f"""<div class="content">"""

		html_string += f"""<table class = "table1"><tr><th>pāli</th><td>{pali2}</td></tr>"""
		html_string += f"""<tr><th>grammar</th><td>{grammar}"""
		text_full += f"{pali}. {grammar}"
		

		if neg != "":
			html_string += f""", {neg}"""
			text_full += f""", {neg}"""

		if verb != "":
			html_string += f""", {verb}"""
			text_full += f""", {verb}"""

		if trans != "":
			html_string += f""", {trans}"""
			text_full += f""", {trans}"""

		if case != "":
			html_string += f""" ({case})"""
			text_full += f""" ({case})"""

		html_string += f"""</td></tr>"""
		html_string += f"""<tr valign="top"><th>meaning</th><td><b>{meaning}</b>"""
		text_full += f""". {meaning}"""

		if lit != "":
			html_string += f"""; lit. {lit}"""
			text_full += f"""; lit. {lit}"""
		html_string += f"""</td></tr>"""

		if root != "":
			html_string += f"""<tr valign="top"><th>root</th><td>{root}<sup>{root_verb}</sup> {root_grp} {root_sign} ({root_meaning})</td></tr>"""
			text_full += f""". root: {root} {root_grp} {root_sign} ({root_meaning})"""

		if root_in_comps != "" and root_in_comps != "0":
			html_string += f"""<tr valign="top"><th>√ in comps</th><td>{root_in_comps}</td></tr>"""
			text_full += f""", {root_in_comps} in comps"""

		if base != "":
			html_string += f"""<tr valign="top"><th>base</th><td>{base}</td></tr>"""
			text_full += f""". base: {base}"""

		if construction != "":
			html_string += f"""<tr valign="top"><th>construction</th><td>{construction}</td></tr>"""
			construction_text = re.sub("<br/>", ", ", construction)
			text_full += f""". constr: {construction_text}"""

		if derivative != "":
			html_string += f"""<tr valign="top"><th>derivative</th><td>{derivative} ({suffix})</td></tr>"""
			text_full += f""". derivative: {derivative} ({suffix})"""

		if pc != "":
			html_string += f"""<tr valign="top"><th>phonetic</th><td>{pc}</td></tr>"""
			pc_text = re.sub("<br/>", ", ", pc)
			text_full += f""". phonetic: {pc_text}"""

		if comp != "":
			html_string += f"""<tr valign="top"><th>compound</th><td>{comp} ({comp_constr})</td></tr>"""
			text_full += f""". compound: {comp} ({comp_constr})"""

		if ant != "":
			html_string += f"""<tr valign="top"><th>antonym</th><td>{ant}</td></tr>"""
			text_full += f""". antonym: {ant}"""

		if syn != "":
			html_string += f"""<tr valign="top"><th>synonym</th><td>{syn}</td></tr>"""
			text_full += f""". synonym: {syn}"""

		if var != "":
			html_string += f"""<tr valign="top"><th>variant</th><td>{var}</td></tr>"""
			text_full += f"""variant: {var}"""

		if comm != "":
			html_string += f"""<tr valign="top"><th>commentary</th><td>{comm}</td></tr>"""
			comm_text = re.sub("<br/>", " ", comm)
			comm_text = re.sub("<b>", "", comm_text)
			comm_text = re.sub("</b>", "", comm_text)
			text_full += f""". commentary: {comm_text}"""

		if notes != "":
			html_string += f"""<tr valign="top"><th>notes</th><td>{notes}</td></tr>"""
			text_full += f""". notes: {notes}"""

		if cognate != "":
			html_string += f"""<tr valign="top"><th>cognate</th><td>{cognate}</td></tr>"""
			text_full += f""". eng congante: {cognate}"""

		if link != "":
			html_string += f"""<tr valign="top"><th>link</th><td><a href="{link}">{link}</a></td></tr>"""
			text_full += f""". link: {link}"""

		if non_ia != "":
			html_string += f"""<tr valign="top"><th>non ia</th><td>{non_ia}</td></tr>"""
			text_full += f""". non IA: {non_ia}"""

		if sk != "":
			html_string += f"""<tr valign="top"><th>sanskrit</th><td><i>{sk}</i></td></tr>"""
			text_full += f""". sanskrit: {sk}"""

		sk_root_mn = re.sub("'", "", sk_root_mn)
		if sk_root != "":
			html_string += f"""<tr valign="top"><th>sk root</th><td><i>{sk_root} {sk_root_cl} ({sk_root_mn})</i></td></tr>"""
			text_full += f""". sk root: {sk_root} {sk_root_cl} ({sk_root_mn})"""

		html_string += f"""</table></div>"""

# examples
					
		if eg1 != "" and eg2 != "":
			html_string += f"""<input type="checkbox" id="title2{pali_}" /><label for="title2{pali_}">examples</label>"""
			html_string += f"""<div class="content">"""
			html_string += f"""<p>{eg1}<br><span style="color:{colour1}"><i>{source1} {sutta1}</i></span>"""
			html_string += f"""<p>{eg2}<br><span style="color:{colour1}"><i>{source2} {sutta2}</i></span>"""
			html_string += f"""</div>"""
			
		elif eg1 != "" and eg2 == "":
			html_string += f"""<input type="checkbox" id="title2{pali_}" /><label for="title2{pali_}">example</label>"""
			html_string += f"""<div class="content">"""
			html_string += f"""<p>{eg1}<br><span style="color:{colour1}"><i>{source1} {sutta1}</i></span>"""
			html_string += f"""</div>"""
					
# inflection table

	if pos not in indeclinables:
		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/output/html/{pali}.html") as table_data:
				table_data_read = table_data.read()

		except:
			error_log.write(f"error reading inflection table - <b>{pali}</b>.html\n")

		if pos in declensions:
			html_string += f"""<input type="checkbox" id="title3{pali_}" /><label for="title3{pali_}">declension</label>"""
		if pos in conjugations:
			html_string += f"""<input type="checkbox" id="title3{pali_}" /><label for="title3{pali_}">conjugation</label>"""
		if pos == "sandhi" or pos == "idiom":
			html_string += f"""<input type="checkbox" id="title3{pali_}" /><label for="title3{pali_}">inflection</label>"""

		html_string += f"""<div class="content">"""
		html_string += f"""{table_data_read}"""

		if pos != "sandhi" and pos != "idiom":
			if pos in declensions:
				html_string += f"""<p>Spot a mistake in the declension table? Something missing? """
				html_string += f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSfKUBx-icfRCWmhHqUwzX60BVQE21s_NERNfU2VvbjEfE371A/viewform?usp=pp_url&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>"""

			if pos in conjugations:
				html_string += f"""<p>Spot a mistake in the declension table? Something missing? """
				html_string += f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSdAL2PzavyrtXgGmtNrZAMyh3hV6g3fU0chxhWFxunQEZtH0g/viewform?usp=pp_url&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>"""
		html_string += f"""</div>"""


# root family

	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/subfamily html/{root} {root_grp} {root_meaning} {family}.html") as table_data:
			table_data_read = table_data.read()

	except:
			if root != "":
				error_log.write(f"""error reading subfamily - {pali} - {root} {root_grp} {root_meaning} {family}\n""")

	if family != "":
		html_string += f"""<input type="checkbox" id="title4{pali_}" /><label for="title4{pali_}">root family</label>"""
		html_string += f"""<div class="content">"""
		html_string += f"""<p><b>{pali_clean}</b> belongs to the root family <b>{family}</b> ({root_meaning})</p>"""
		html_string += f"""<table class = "table1">{table_data_read}</table>"""
		html_string += f"""</div>"""

# compound families
	with open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/compound_family_list.csv", "r") as cfl: 
		cfl = cfl.read()
	
	if family2 != "" and meaning != "":
		html_string += f"""<input type="checkbox" id="title5{pali_}" /><label for="title5{pali_}">compound family</label>"""
		html_string += f"""<div class="content">"""
	
		compound_family_list = []
		compound_family_list = list(family2.split())

		for cf in compound_family_list:
				try:
					with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{cf}.html", "r") as data:
						data_read = data.read()
						html_string += f"""<p>compounds which contain <b>{cf}</b></p>"""
						html_string += f"""<table class = "table1">{data_read}</table>"""
						
				except:
					if family2 != "":
								error_log.write(f"""error reading compound family - {pali} - {cf}\n""")

		html_string += f"""</div>"""

	if family2 == "" and meaning != "" and re.search(fr"\b{pali_clean}\b", cfl):
		html_string += f"""<input type="checkbox" id="title5{pali_}" /><label for="title5{pali_}">compound family</label>"""
		html_string += f"""<div class="content">"""

		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{pali_clean}.html", "r") as data:
				data_read = data.read()
				html_string += f"""<p>compounds which contain <b>{pali_clean}</b></p>"""
				html_string += f"""<table class = "table1">{data_read}</table>"""
				
		except:
			if family2 != "":
						error_log.write(f"""error reading compound family - {pali} - {family2}\n""")
		html_string += f"""</div>"""

# feedback
	html_string += f"""<input type="checkbox" id="title6{pali_}" /><label for="title6{pali_}">feedback</label>"""
	html_string += f"""<div class="content">"""
	html_string += f"""<p>Did you spot a mistake in the dictionary? Have a bright idea?  """
	html_string += f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSf9boBe7k5tCwq7LdWgBHHGIPVc4ROO5yjVDo1X5LDAxkmGWQ/viewform?usp=pp_url&entry.1433863141=GoldenDict {today}" target="_blank">Please report it here.</a>"""
	html_string += f"""<span> Your corrections and suggestions help to improve the quality of this dictionary for everyone.</span>"""

	html_string += f"""<p>You are using DPD GoldenDict generated on {today} Please <a class="link" href="https://drive.google.com/drive/folders/1W66OXwX6KQHnK0b-7E1enZJQLjXehit4?usp=sharing">check here</a> every full moon for an update. Or email <a class="link" href = "mailto:digitalpalidictionary@gmail.com?subject=Update me!&body=Please send me DPD updates as soon as they're available.">digitalpalidictionary@gmail.com</a> to get notified of updates as soon as they become available.</p>"""
	html_string += f"""</div>"""
	html_string += f"""</body></html>"""

# write gd.json
	
	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/output/synonyms/{pali}", "rb") as syn_file:
			synonyms = pickle.load(syn_file)
			synonyms = (synonyms)

	except:
		error_log.write(f"error reading synonyms - {pali}\n")
		print(f"error reading synonyms - {pali}")

# data compiling

	html_data_list += [[f"{pali}", f"""{html_string}""", "", synonyms]]
	text_data_full += f"{text_full}\n"
	text_data_concise += f"{text_concise}\n"

	if row % 100 == 0:
		with open(f"output/html/{pali} (sample).html", "w", encoding ="utf8") as text_file:
			text_file.write(html_string)

error_log.close()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating html & json files
0/44962	akakkasa
5000/44962	ambila 2
10000/44962	ubhayathā
15000/44962	guṇa 3
20000/44962	duggandha 1
25000/44962	paṭisevi
30000/44962	buddhālambana
35000/44962	vajjetvā
40000/44962	sandhiccheda 1


write text versions

In [61]:
with open("share/dpd_full.txt", "w", encoding ="utf8") as text_file:
	text_file.write(text_data_full)

with open("share/dpd_concise.txt", "w", encoding ="utf8") as text_file:
	text_file.write(text_data_concise)


html list > dataframe

In [62]:
pali_data_df = pd.DataFrame(html_data_list)
pali_data_df.columns=["word", "definition_html", "definition_plain", "synonyms"]

generate roots html

In [63]:
print("~" * 40)
print("generating roots html & json")
print("~" * 40)

root_data_list = []

html_style2 = """

	table.root_table td,table.root_table th 
	{ border:2px solid #fff;
	line-height:150%;
	margin:5px 10px;
	padding:2px 5px;
	text-align:left;
	transition-duration: .2s;
	vertical-align:top;
	}
	
	table.root_table th 
	{ color:#AB3C00;
	font-weight:700;
	white-space:nowrap;
	}
	
	table.root_table td:hover , table.root_table th:hover
	{ background-color:#F95700;
	}
	
	div.root_content 
	{ border:2px solid #AB3C00;
	border-radius:5px;
	line-height:150%;
	overflow:auto;
	text-align:left;
	}

	label.root
	{ background-color:#FFE2D2;
	border-radius:10px;
	box-shadow:0 4px 4px -2px #888888;
	color:#000;
	cursor:pointer;
	display:inline-block;
	font-size:80%;
	font-weight:400;
	margin:3px 0;
	padding:5px;
	transition-duration: .2s;
	}
	
	label.root:hover, input:checked + label.root:hover 
	{ background-color:#AB3C00;
	color:white;
	}

	input:checked + label.root 
	{ background-color:#F95700;
	}

	input + label + div.root_content 
	{ display:none;
	}

	input:checked + label + div.root_content 
	{ display:block;
	}
	"""

for row in range(roots_df_length):

	html_string = ""

	root_count = roots_df.iloc[row,1]
	root = roots_df.iloc[row, 2]
	root_in_comps = roots_df.iloc[row, 3]
	root_has_verb = roots_df.iloc[row, 4]
	root_group = roots_df.iloc[row, 5]
	root_sign = roots_df.iloc[row, 6]
	root_meaning = roots_df.iloc[row, 8]
	root_meaning_ = re.sub(" ", "_", root_meaning)

	if root_count != "0":
		
		if row % 100 == 0 or row % roots_df_length == 0:
			print(f"{row}/{roots_df_length}\t{root} {root_group} {root_meaning}")

	# header

		html_string += """<!DOCTYPE html><html><head><meta charset="utf-8"></head><style>"""
		html_string += html_style
		html_string += html_style2
		html_string += "</style><body>"
		html_string += f"""<div class="root_content"><p>root. <b>{root}</b><sup>{root_has_verb}</sup>{root_group} {root_sign} ({root_meaning})</p></div>"""

	# root info
		html_string += f"""<input type="checkbox"  id="root info{root}_{root_group}_{root_meaning_}"/><label class="root" for="root info{root}_{root_group}_{root_meaning_}">root info</label>""" ### edit!!!
		html_string += f"""<div class="root_content">"""

		root_info = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/root info/{root} {root_group} {root_meaning}.html", "r")
		root_info_read = root_info.read()
		root_info.close

		html_string += f"""<table class="root_table">{root_info_read}</table>"""
		html_string += """</div>"""

# root families

		root_families_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/families/{root} {root_group} {root_meaning}.csv", sep="\t", header=None)

		root_families_length = root_families_df.shape[0]

		if root_families_length < 0:
			pass
		else:
			for line in range(root_families_length):
				subfamily = root_families_df.iloc[line, 0]

				html_string += f"""<input type="checkbox"  id="rootfam{root}_{root_group}_{root_meaning_}_{subfamily}" /><label class="root" for="rootfam{root}_{root_group}_{root_meaning_}_{subfamily}">{subfamily}</label>"""
				html_string += """<div class="root_content">"""
				html_string += f"""<p>all words which belong to the root family <b>{subfamily}</b></p>"""

				try:
					subfamily_html = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/subfamily html/{root} {root_group} {root_meaning} {subfamily}.html")
					subfamily_html_read = subfamily_html.read()
					html_string += f"""<table class="root_table">{subfamily_html_read}</table>"""
					
				except:
					print(f"error\t{root} {root_group} {root_meaning} {subfamily}.html")

				html_string += "</div>"

		html_string += "</body></html>"

		if row % 50 == 0:
			root_html = open(f"output/root html/{root} {root_group} {root_meaning} (sample).html", "w")
			root_html.write(html_string)

		root_clean = re.sub("√", "", root)
		synonyms = [root,root_clean]

		# compile root data into list
		root_data_list += [[f"{root}", f"""{html_string}""", "", synonyms]]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating roots html & json
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0/1011	√acc 1 shine, praise
100/1011	√cāy 1 honour
200/1011	√du 1 torment, afflict
600/1011	√pac 1 cook
800/1011	√sinā 1 bathe
1000/1011	√yas 3 strive, struggle


roots > dataframe > json

In [64]:
root_data_df = pd.DataFrame(root_data_list)
root_data_df.columns=["word", "definition_html", "definition_plain", "synonyms"]

pali_data_df = pali_data_df.append(root_data_df)
pali_data_df.to_json("output/gd.json", force_ascii=False, orient="records", indent=6)

generate goldendict

In [65]:
print("~" * 40)
print("generating goldendict")



from typing import List
from pathlib import Path
from simsapa.app.stardict import export_words_as_stardict_zip, ifo_from_opts, DictEntry
import json

zip_path = Path("./dpd_test.zip")

with open("output/gd.json", "r") as gd_data:
    gd_data_read = json.load(gd_data)

def item_to_word(x):
    return DictEntry(
        word=x["word"],
        definition_html=x["definition_html"],
        definition_plain=x["definition_plain"],
        synonyms=x["synonyms"],
    )

words = list(map(item_to_word, gd_data_read))

icon_path = Path("/home/bhikkhu/Bodhirasa/Dropbox/dpd/favicon/favicon_io nu circle/favicon.ico")

ifo = ifo_from_opts(
    {
        "bookname": "dpd",
        "author": "bdhrs",
        "description": "digital pāli dictionary",
        "website": "https://github.com/bdhrs",
    }
)

export_words_as_stardict_zip(words, ifo, zip_path, icon_path)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating goldendict


In [67]:
print("~" * 40)
print("copying goldendict")
os.popen(f'mv "/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/dpd_test.zip" "dpd_test_{today}.zip"')
os.popen(f'mv -u "dpd_test_{today}.zip" "share/dpd_test_{today}.zip"')
os.popen(f'unzip -o "/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/share/dpd_test*.zip" -d "/home/bhikkhu/Documents/Golden Dict"')
print("~" * 40)
print("fin")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
copying goldendict
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
fin


mv: cannot stat '/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/dpd_test.zip': No such file or directorymv: 
cannot stat 'dpd_test_2022-01-08.zip': No such file or directory

2 archives were successfully processed.
